In [10]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Mobile_Report

### Links:

[[total_active_users_per_day_full_table]]
___

In [13]:
query_text = """--sql
CREATE TABLE db1.maf_rep_mobile_total
(
    `report_date` Date,
    `MAF` UInt64,
    `partner_uuid` String,
    `city` String,
    `stricted_MAF` UInt64,
    `freemonetization_MAF` UInt64
)
ENGINE = MergeTree()
ORDER BY report_date
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [15]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.maf_rep_mobile_total_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 45 MINUTE TO db1.maf_rep_mobile_total AS
    SELECT
        partner_uuid,
        city,
        report_date AS report_date,
        COUNT(DISTINCT flat_uuid) AS MAF,
        COUNT(DISTINCT if(monetization = 0, flat_uuid, null)) AS stricted_MAF,
        COUNT(DISTINCT if(monetization = 1 or monetization is null, flat_uuid, null)) AS freemonetization_MAF
    FROM db1.total_active_users_per_day_full_table
    GROUP BY report_date,
        partner_uuid,
        city
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [7]:
query_text = """--sql
    SELECT
        *
    FROM db1.maf_rep_mobile_total
    ORDER BY report_date desc
    limit 10
    """

ch.query_run(query_text)


shape: (0, 0)
┌┐
╞╡
└┘

### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.maf_rep_mobile_total DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [11]:
query_text = """--sql
    DROP TABLE db1.maf_rep_mobile_total_mv
    """

ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 60
 Code: 60. DB::Exception: Table db1.maf_rep_mobile_total_mv does not exist. (UNKNOWN_TABLE) (version 25.3.3.42 (official build))


### drop table

In [9]:
query_text = """--sql
    DROP TABLE db1.maf_rep_mobile_total
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.maf_rep_mobile_total_mv
"""

ch.query_run(query_text)